In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import random
import os
import time
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
train_targets_not_scored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
                                       
                            

In [ ]:
train_features.head()

In [ ]:
train_features['cp_dose'].value_counts()

In [ ]:
cat_cols = ['cp_dose', 'cp_type']
train_features_oe = pd.get_dummies(train_features, columns=cat_cols)

In [ ]:
test_features_oe = pd.get_dummies(test_features, columns=cat_cols)

In [ ]:
train_features.shape

In [ ]:
def model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(877),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4056, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4056, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(512, activation="relu"),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(512, activation="relu"),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(512, activation="sigmoid"),
#         tf.keras.layers.Dense(256, activation="relu"),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(256, activation="relu"),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(206, activation="sigmoid")
        ])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2.75e-5), loss='binary_crossentropy', metrics=["accuracy", "AUC"])
    return model

In [ ]:
model_test = model()
model_test.summary()

In [ ]:
train = train_features_oe.drop('sig_id', 1).to_numpy()
test = test_features_oe.drop('sig_id', 1).to_numpy()
targets = train_targets.drop('sig_id', 1).to_numpy()

In [ ]:
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', 
                                   factor=0.1, patience=3, verbose=1, 
                                   epsilon=1e-4, mode='min')

In [ ]:
ReduceLROnPlateau?

In [ ]:
from sklearn.model_selection import KFold
start_time = time.time()
NFOLD = 5
kf = KFold(n_splits=NFOLD)

BATCH_SIZE=512
EPOCHS=15
train_predictions = np.zeros((train_features_oe.shape[0], 206))
test_predictions = np.zeros((test_features_oe.shape[0], 206))
models = []
for fold, (train_idx, valid_idx) in enumerate(kf.split(train)) :
    print(f"FOLD{fold}")
#     ctl_mask = train[:,0]=='ctl_vehicle'
#     train  = train[~ctl_mask, :]
#     targets = targets[~ctl_mask]
    net = model()
    #fit the model
    net.fit(train[train_idx], targets[train_idx], batch_size=BATCH_SIZE, epochs=EPOCHS,
           validation_data = (train[valid_idx], targets[valid_idx]), verbose=1, callbacks=[reduce_lr_loss])
    print("train")
    print(list(zip(net.metrics_names, net.evaluate(train[train_idx], targets[train_idx], 
                                                   batch_size=BATCH_SIZE, verbose=0))))
    print("Valdiation")
    print(list(zip(net.metrics_names, net.evaluate(train[valid_idx], targets[valid_idx], 
                                                   batch_size=BATCH_SIZE, verbose=0))))
    print("Train Predictions")
    train_predictions[train_idx] = net.predict(train[train_idx], batch_size=BATCH_SIZE , verbose=0)
    train_predictions[valid_idx] = net.predict(train[valid_idx], batch_size=BATCH_SIZE , verbose=0)
    test_predictions += net.predict(test, batch_size=BATCH_SIZE , verbose=0)/NFOLD
print(f"Time taken is{round((time.time() - start_time)/60)} minutes")
    

In [ ]:
sub =pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
sub.iloc[:,1:] = test_predictions
sub.to_csv('submission.csv', index=False)